# Setup & Bibliotheken

In [ ]:
'''# Schritt 1: Deinstalliere alle problematischen Pakete
!pip uninstall tensorflow tensorflow-text tf-keras tensorflow-decision-forests tensorflow-probability -y

# Schritt 2: Installiere kompatible Versionen
!pip install tensorflow==2.19.0 tensorflow-probability==0.25.0 tensorflow-text==2.19.0 tf-keras==2.19.0

!pip install -q sentence-transformers transformers accelerate torch pandas tqdm

# Schritt 3: Unterdrücke Warnungen (für parakeet und CUDA)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Unterdrückt TensorFlow-Warnungen

# Schritt 4: Teste den Import
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
print("Import erfolgreich!")

# Schritt 5: Teste tensorflow_probability
import tensorflow_probability as tfp
print(f"TensorFlow Probability Version: {tfp.__version__}")
!pip install transformers torch pandas tqdm -q'''

'# Schritt 1: Deinstalliere alle problematischen Pakete\n!pip uninstall tensorflow tensorflow-text tf-keras tensorflow-decision-forests tensorflow-probability -y\n\n# Schritt 2: Installiere kompatible Versionen\n!pip install tensorflow==2.19.0 tensorflow-probability==0.25.0 tensorflow-text==2.19.0 tf-keras==2.19.0\n\n!pip install -q sentence-transformers transformers accelerate torch pandas tqdm\n\n# Schritt 3: Unterdrücke Warnungen (für parakeet und CUDA)\nimport os\nos.environ[\'TF_CPP_MIN_LOG_LEVEL\'] = \'2\'  # Unterdrückt TensorFlow-Warnungen\n\n# Schritt 4: Teste den Import\nfrom transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline\nprint("Import erfolgreich!")\n\n# Schritt 5: Teste tensorflow_probability\nimport tensorflow_probability as tfp\nprint(f"TensorFlow Probability Version: {tfp.__version__}")\n!pip install transformers torch pandas tqdm -q'

In [8]:

import platform #infos aus dem System verarbeiten können (Windows)
import os #für Dateihändelung
import pandas as pd #für Datenanalyse (Bereinigung von Daten)
import random
from tqdm import tqdm #um ausgeben zu können, wie weit die Prozesse laufen (für Prozess Balken)
import tensorflow_probability as tfp # Wahrscheinlichekti von Ergebnissen analysieren & bewerten
import torch # soll dabei helfen, schnellere Berechnungen zu machen (GPU)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM #aus hugging face bibliothek, um Texte in Tokens umzuwandeln, Textsequenzen als Input und andere Textsequenzen ausgeben (Output)
#Aufgabe: Übersetzen, Zusammenfassen, Generieren
from sentence_transformers import SentenceTransformer, util #sentenceTranformer, damit werden Modelle geladen, die Texte in ganze Vektoren umwandeln
#ür Sätze/Texte erzeugen → Ähnlichkeitsvergleiche
from google.colab import files, drive #Interaktion mit lokalen Dateien in Google Colab Umgebung
from datetime import datetime# Manipulation von Zeitangaben
import shutil #Operationen mit Dateien und mit dem Sammeln von Dateien
import re #Operationen für reguläre Ausdrücke (regex)
import json
import math
#from google.colab import drive



In [2]:
def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

# Gewählte xlsl Datei in csv Datei umwandeln und umbenennen

In [ ]:
# @title

read_file = pd.DataFrame(pd.read_excel('/content/drive/My Drive/Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).xlsx'))
read_file.to_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv", index = False, header = True, encoding=smartEncoding())
df = pd.DataFrame(pd.read_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv"))
#print(df.head)

In [ ]:
# @title
#wähle relevante Spalten
gewaehlte_spalten = df[["Erfahrungsbericht des Nutzers" , "Zufallszahl"]]

#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv", index = False)




# Datenbereinigung

In [ ]:
erfahrungen_gefiltert = pd.read_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_gefiltert.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_gefiltert.drop_duplicates(inplace=True)
#speichern
erfahrungen_gefiltert.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index=False)# Entfernung von Duplikaten wurde beim df selbst vollzogen anstatt als Kopie
print(f"FINAL: {len(erfahrungen_gefiltert)} Reihen")


#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)


#zufällig n = 100 Berichte für erfahrungen_final.csv auswählen
erfahrungen_clean = pd.read_csv("/content/drive/My Drive/erfahrungen_clean.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_clean.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_clean.drop_duplicates(inplace=True)
erfahrungen_clean.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)

# INDEX-Nummern zufällig wählen
zufalls_index = random.sample(range(len(erfahrungen_clean)), k=100)
zufall_100 = erfahrungen_clean.iloc[zufalls_index]



#speichern (NUR 100!)
zufall_100.to_csv("/content/drive/My Drive/zufall_100_berichte.csv", index=False)

print(f"{len(zufall_100)} Zufallsberichte!")

FINAL: 6453 Reihen
100 Zufallsberichte!


# **Datenaugmentation und Syn-Chain-Methode**

## Datenaugmentation mit Trainingsdaten (ca. 11.900 Rezensionen)

## Paraphrasieren

In [ ]:

# Google Drive mounten/ Ordner in Google Colab Umbegung erstellen lassen (einmalige Ausführung nötig)
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/MyDrive/ABSA_Paraphrase/' #Unterordner "ABSA_Paraphrase" in Google Colab Umgebung erstellen
os.makedirs(SAVE_PATH, exist_ok=True) #Methode, wobei ein Projektverzeichnis (Pfad für Paraphrase mit inbegriffen) festgelegt wird

print("Ordner & Bibliotheken geladen!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ordner & Bibliotheken geladen!


Beste Lösung bislang

In [3]:
def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

# Paraphrasierung

# Erstellung eines "Synonym-Dictionarys", zuerste mit Schlüsselwert wie z.B. "schmutzig" & danach Wertpaaren, Duplikate nicht erlaubt
SYNONYMS = {
    "schmutzig": ["dreckig", "unhygienisch", "unrein" , "unsauber", "verschmutzt", "versifft", "gammelig", "schmuddelig"],
    "hygienisch": ["sauber", "rein", "keimfrei", "gesäubert", "picobello", "blitzeblank"],
    "arbeite": ["bin tätig", "beschäftigt", "bin angestellt"],
    "seit": ["seit etwa", "schon seit", "seit circa"],
    "im": ["in", "bei"],
    "und": ["sowie", "–", "&"],
    "Mutter": ["familienangehörige"],
    "Vater": ["familienangehöriger"],
    "Großvater": ["opa"],
    "Großmutter": ["oma"],
    "waren": ["lebten", "wohnten"],
    "dort": ["dort", "im Heim", "in der Einrichung"],
    "ist": ["bleibt"],
    "sehr": ["wirklich", "besonders"],
    "kollegial": ["freundschaftlich", "kameradschaftlich", "wie ein guter Kollege"],
    "Betreuung": ["pflege", "fürsorge", "hilfe", "obhut", "unterstützung"],
    "Bewohner": ["senioren", "pflegebedürftige"],
    "Essen": [ "Speisen", "Nahrung"],
    "Fest": ["feier", "party" "fete"],
    "beliebt": ["geschätzt", "begehrt", "wird gemocht"],
    "schlecht": ["mies", "mieserabel", "schrecklich" "mangelhaft", "unkorrekt", "daneben"],
    "warum": ["weshalb", "wieso"],
    "toll": ["super", "top", "einwandfrei", "hammer"],
    "tolle": ["einwandfreie", "hammermäßige"],
    "sauber": ["ordentlich", "gepflegt"],
    "frische": ["angenehme"],
    "lecker": ["köstlich", "schmackhaft", "gut"],
    "okay": ["in Ordnung", "befriedigend"],
    "traurig": ["schade", "deprimierend"],
    "Pflegeheim": ["stationäre Pflegeheim"],
    "Haus": ["Heim"],
    "Einrichtung": ["Institution"],
    "freundlich": ["höflich", "zuvorkommend"],
    "unhöflich": ["unsensibel", "nicht nett", "unfreundlich"],
    "kompetent": ["fähig", "professionell", "tüchtig", "effektiv"],
    "nett": ["herzlich", "symphatisch", "angenehm"],
    "nette": ["angenehme", "symphatische", "gute"],
    "nettes": ["angenehmes", "symphatisches", "herzliches"],
    "Oma": ["großmutter", "Omi"],
    "Opa": ["Großvater", "Opi"],
    "Ahnung": ["kenntniss", "Wissen" ],
    "Vorher": ["davor", "damals"],
    "Mitarbeiter": ["beschäftigten", "pflegenden", "angestellten"],
    "motiviert": ["engagiert", "eifrig", "arbeitsfreudig"],
    "Arbeitsklima": ["Klima bei der Arbeit"],
    "Katastrophe": ["Zumutung"]

}# kann noch erweitert werden!


def synonym_replace(text): #nimmt Text an einer bestimmten Stelle aus Spalte "Erfahrungsberichts des Nutzers" aus erfahrungen_clean.csv entgegen
    words = text.split() # Variable, in welches einzelne Wörter in text als Tokens in eine Liste gepackt werden ["wort1", "wort2",...]
    new_words = [] #neue liste erstellen
    for w in words: #schleife
        lower = w.lower().rstrip('.,!?') # buchstaben in kleine Buchstaben schreiben, Satzzeichen vom Ende des Strings entfernen
        if lower in SYNONYMS and random.random() < 0.5: #wenn Wort in dict Synonym vorhanden ist
        #: nicht jedes einzelne Wort ersetzen; Nur in 50% der Fälle ein Synonym ersetzen
            repl = random.choice(SYNONYMS[lower]) #nicht jedes einzelne Wort ersetzen; Nur in 50% der Fälle ein Synonym ersetzen
            if w[0].isupper(): repl = repl.capitalize()#wenn der erste Buchstabe von w groß ist (True/False):
              #Schreibe den ersten Buchstaben des Synonym auch groß!
            new_words.append(repl + w[len(lower):]) #füge in leere Liste das Synonym, mit ersten Buchstaben und restlichen kleingeschriebenen
            #print(w[len(lower):])
        else: #wenn anfangsbuchstabe nicht groß ist
            new_words.append(w) #füge zur Liste das ganze Wort hinzu
    return ' '.join(new_words) #Rückgabe der Elemente von Liste & mache diesen zu einem Satz mit Leerzeichen als Seperator

#funktion für Parahrase
def hybrid_paraphrase(text, n=3):
    original = str(text).strip()
    if len(original) < 20:
        return [original] * n

    results = set()
    max_attempts = n * 5  # max. 15 Versuche

    for _ in range(max_attempts):
        if len(results) >= n:#wenn der Satz länger als drei Wörter lang ist, stop diese Schleife!
            break

        step1 = synonym_replace(original)#nimm Originaltext aus Zeile von "Erfahrungsbericht des Nutzers" als Parameter
        if len(step1.split()) > 10 and random.random() < 0.3:
            sentences = re.split(r'(?<=[.!?])\s+', step1)
           # print(sentences) Beispiel: ['Ein sehr schönes Haus mit altem Charme, ländlich gelegen.', '...]
            if len(sentences) > 1:
                random.shuffle(sentences)# um die Satzstruktur zu verändern, wenn der Satz lang genug ist
                step1 = ' '.join(sentences)# geshuffelte sentences zum ganzen Satz machen
        results.add(step1)#ist noch ein set z.B. {'Die Vergabe von einem od 2 Sternen kann....'}

    # garantierte Ausgabe,damit Prgramm an dieser schleife nicht hängen bleibt
    result_list = list(results)
    while len(result_list) < n:
        result_list.append(original)

    return result_list[:n]

# erfahrungen_clean.csv laden, falls noch nicht vorhanden
try:
    df
except NameError:
    df = pd.read_csv("/content/drive/MyDrive/erfahrungen_clean.csv") # lieber direkten Pfad aufschreiben
    df = df.dropna(subset=['Erfahrungsbericht des Nutzers']).reset_index(drop=True)
    print(f"{len(df)} Rezensionen geladen!")

# Test mit 10 Rezensionen aus erfahrungen_clean.csv
test_indices = random.sample(range(len(df)), 10)
print(f"\n10-TEST: HYBRID \n")
print("="*90)

for i, idx in enumerate(test_indices, 1):
    text = str(df.iloc[idx]['Erfahrungsbericht des Nutzers'])
    print(f"\n{i}. ORIGINAL (Index {idx}):")
    print(text[:200] + ("..." if len(text) > 200 else ""))
    print("\n   PARAPHRASEN:")
    paras = hybrid_paraphrase(text, 3)
    for j, p in enumerate(paras, 1):
        print(f"   {j}: {p[:200]}{'...' if len(p) > 200 else ''}")
    print("-" * 90)

6453 Rezensionen geladen!

10-TEST: HYBRID 


1. ORIGINAL (Index 2433):
Eine tolle Einrichtung mit liebevoller Betreuung und wunderbarem Garten. Dazu gibt es nebenan die Möglichkeit des Betreuten Wohnens. Gutes Seniorenheim!

   PARAPHRASEN:
   1: Eine hammermäßige Einrichtung mit liebevoller Betreuung & wunderbarem Garten. Dazu gibt es nebenan die Möglichkeit des Betreuten Wohnens. Gutes Seniorenheim!
   2: Eine hammermäßige Einrichtung mit liebevoller Betreuung – wunderbarem Garten. Dazu gibt es nebenan die Möglichkeit des Betreuten Wohnens. Gutes Seniorenheim!
   3: Gutes Seniorenheim! Dazu gibt es nebenan die Möglichkeit des Betreuten Wohnens. Eine hammermäßige Einrichtung mit liebevoller Betreuung und wunderbarem Garten.
------------------------------------------------------------------------------------------

2. ORIGINAL (Index 5739):
Sehr schöne Einrichtung! Nettes Pflegepersonal! Tolle Einrichtungsleitung !

   PARAPHRASEN:
   1: Sehr schöne Einrichtung! Angenehmes Pflegeperso

In [ ]:
# finaler Batch: Alle Rezensionen aus "efahrungen_clean.csv" augmentieren
print(f"\nSTART: Paraphrasierung aller {len(df)} Rezensionen -> {len(df)*3} Samples...")
aug_data = []

for idx in tqdm(range(len(df)), desc="Augmentiere", unit="text"):
    text = df.iloc[idx]['Erfahrungsbericht des Nutzers']
    zufallszahl = int(df.iloc[idx]['Zufallszahl'])
    aug_data.append({
        'id': zufallszahl,
        'original': text
    })
    variants = hybrid_paraphrase(text, 3)
    for v_id, para in enumerate(variants, 1):
      aug_data.append({
          'variant_id': v_id,
          'paraphrase': para
      })

# Speichern
with open("data.jsonl", "w", encoding="utf-8") as f:
    for group in aug_data:
        f.write(json.dumps(group, ensure_ascii=False) + "\n")

aug_df = pd.DataFrame(aug_data)
output_path = "/content/drive/MyDrive/ABSA_Paraphrase/augmented_erfahrungen.csv"
aug_df.to_csv(output_path, index=False)

print(f"\nFERTIG!")
print(f"→ {len(aug_df)} Paraphrasen gespeichert")
print(f"→ Datei: {output_path}")
print(f"→ Robuster Datensatz für Grok 3 Mini bei der ABSA vorbereitet!")


START: Paraphrasierung aller 6453 Rezensionen -> 19359 Samples...


Augmentiere: 100%|██████████| 6453/6453 [00:01<00:00, 3312.64text/s]



FERTIG!
→ 25812 Paraphrasen gespeichert
→ Datei: /content/drive/MyDrive/ABSA_Paraphrase/augmented_erfahrungen.csv
→ Robuster Datensatz für Grok 3 Mini bei der ABSA vorbereitet!


Nan-Werte entfernen aus JSONL-Datei

In [9]:

input_file = "/content/sample_data/data_training.jsonl"
output_file = "cleaned_jsonl.jsonl"

def find_nan(obj):
    if isinstance(obj, float):
        return math.isnan(obj) or math.isinf(obj)
    if isinstance(obj, dict):
        return any(find_nan(v) for v in obj.values())
    if isinstance(obj, list):
        return any(find_nan(v) for v in obj)
    return False

with open(input_file, 'r', encoding='utf-8') as f, \
     open(output_file, 'w', encoding='utf-8') as fi:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
            if not find_nan(obj):
                fi.write(json.dumps(obj, ensure_ascii=False) + '\n')
        except json.JSONDecodeError:
            pass  # skip invalid lines


für Fine Tuning:
Hugging Face: Fine-tune ein Grok-ähnliches Modell (z. B. "xai-org/grok-1" oder "reedmayhew/Grok-3-gemma3-4B-distilled" als Distillation von Grok 3). Das ist open-source und GitHub-freundlich.

Code-Beispiel (Distillation von Grok 3 via API + Fine-Tuning auf Gemma-3 4B):


Best Practices (aus Reddit, GitHub Blog, HF Docs):

Lizenz: Füge eine LICENSE (z. B. Apache 2.0) hinzu, da Grok-Modelle proprietär sind – dein Fine-Tuning erbt das.
Model Card: Erstelle eine (HF-Template: https://huggingface.co/docs/hub/model-cards) mit Details zu Daten (12.000 Rezensionen), Augmentation und Metriken.
Datenschutz: Anonymisiere sensible Daten in der CSV (z. B. Namen entfernen), bevor du hochlädst – Pflegeberichte könnten personenbezogen sein (DSGVO-konform!).
Versionskontrolle: Nutze GitHub Releases für große Dateien (z. B. Weights als Asset).
Integration mit HF: Push zu Hugging Face Hub und verlinke im GitHub-Repo – HF ist GitHub-ähnlich und unterstützt Grok-Destillationen (z. B. "reedmayhew/Grok-3-gemma3-4B-distilled").



3. Empfehlung für deine Bachelorarbeit

Starte mit Option A/B: Prompt Engineering für schnelle Tests, dann Hugging Face für echtes Fine-Tuning (ca. 30–60 Min. in Colab mit GPU).
GitHub-Repo als Portfolio: Lade Code, Skripte und Metriken hoch – das zeigt Reproduzierbarkeit. Vermeide rohe Weights (>100 MB) direkt; verlinke sie.
Nächste Schritte: Hole dir einen xAI API-Key und HF-Token. Teste den Code in Colab.

Falls du Hilfe beim Setup (z. B. API-Key, spezifischer Code-Fehler) brauchst oder mehr Details zu einer Option, lass es mich wissen!

In [ ]:
# um damit zu beginnen: https://arxiv.org/html/2507.09485